In [1]:
import pandas as pd
import csv
import numpy as np
from random import shuffle
import funcs as fc
import decimal

In [2]:
ctx = decimal.Context()
ctx.prec = 20

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

def getTwoNumbersGap(num1, num2):
    return abs(num1 - num2) + 1

def getRangeGap(data):
    maxV = max(data)
    minV = min(data)
    return getTwoNumbersGap(maxV, minV)

def getMinMaxFromMultiD(sourceList):
    maxValue = max(sourceList.reshape(-1))
    minValue = min(sourceList.reshape(-1))
    
    return maxValue, minValue

def getCategoryDiversityCNT(sourceList):
    return len(set([j for sub in sourceList for j in sub]))

In [3]:
def getOrgAllColumnDivertyCNT(datapath):
    org_moned_dataset = fc.readCSV(datapath, sep='\t')
    print(org_moned_dataset[0])
    org_moned_dataset = org_moned_dataset[1:]
    
    for dataRow in org_moned_dataset:
        dataRow[0] = int(dataRow[0])
        # 원본데이터 각 컬럼별 데이터 diversity count 추출
    
    columnsDiversityCNT = dict()
    
    for i, colName in enumerate(evalColNames):
        if colName in categoricalColNames:
            colIDX = evalColNames.index(colName)
            columnsDiversityCNT[colIDX] = len(set([data[i] for data in org_moned_dataset]))
        else:
            diversityCNT = getRangeGap([float(data[i]) for data in org_moned_dataset])
            columnsDiversityCNT[i] =diversityCNT       
        
        
    return columnsDiversityCNT
    

def getCatetoryColDiversityCNT(cellData):
    cnt = len(cellData.split(','))
    return cnt

def getNumericColDiversityCNT(cellData):
    numericData = [float(data.strip()) for data in cellData.split('-') if data.strip()]
    return getRangeGap(numericData)  

In [4]:
def getLEScore(normalizedDataSet): 
        
    eachColumnLEScoreList = list()    
    normalizedDataSet_cols =normalizedDataSet[0]
    print(normalizedDataSet_cols)
    totalColNum = len(normalizedDataSet_cols)-1
    
    for i, colName in enumerate(normalizedDataSet_cols):
        if colName not in evalColNames: continue
            
        colDataSet = [data[i] for data in normalizedDataSet[1:]]
        isCategoriCol = colName in categoricalColNames
        colDiversityCNT  = evalColDirvertyDIC[evalColNames.index(colName)]   
        
        colLEScoreList = list()  
        for i, cellData in enumerate(colDataSet):
            if isCategoriCol:
                cellDiversityCNT = getCatetoryColDiversityCNT(cellData)
                
            else:
                try:
                    cellDiversityCNT = getNumericColDiversityCNT(cellData)
                except:
                    print(i, cellData)
                    print(colName)
                    break
                
            
            
            colLEScoreList.append(abs(cellDiversityCNT-1)/abs(colDiversityCNT-1))
            
        colLEScore = sum(colLEScoreList) / len(colDataSet)
        print(colName ,"sum :", sum(colLEScoreList))
        print(colName ,"len :", len(colDataSet))
        print(colName ,"colLEScore :", colLEScore)
        print('')
        eachColumnLEScoreList.append(colLEScore)
        
    leScore = sum(eachColumnLEScoreList)/totalColNum 
    leScore = round(leScore, 7)
    print("totla LE SCORE :", leScore)
    
    return leScore

#### CONST VARs

In [5]:
evalColNames = ['AGE', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITION']
categoricalColNames = ['RACE',  'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITION']
originalDataSet_Path = './data/finalPatientDataSet_20.csv'

In [6]:
origin_dataset = fc.readCSV(originalDataSet_Path, sep='\t')

In [7]:
print(len(origin_dataset))

224886


In [8]:
len(set([data[-2] for data in origin_dataset]))

134

In [9]:
deathData = [data for data in origin_dataset if data[-1] == '1']
len(deathData)

29740

In [10]:
freqDict = dict()
for data in deathData:
    if data[-2] not in freqDict:
        freqDict[data[-2]] = 0
    freqDict[data[-2]] += 1
    
sorted(freqDict.items(), key=lambda x:x[1], reverse = True)

[('Chronic congestive heart failure (disorder)', 4729),
 ('Myocardial Infarction', 3620),
 ('Viral sinusitis (disorder)', 3223),
 ('Stroke', 1900),
 ('Acute viral pharyngitis (disorder)', 1746),
 ('Pneumonia', 1658),
 ('Acute bronchitis (disorder)', 1498),
 ('Seizure disorder', 958),
 ('Bullet wound', 896),
 ('Cardiac Arrest', 891),
 ('Non-small cell carcinoma of lung  TNM stage 1 (disorder)', 471),
 ('Concussion injury of brain', 411),
 ('Neoplasm of prostate', 386),
 ('Streptococcal sore throat (disorder)', 346),
 ('Otitis media', 289),
 ('Drug overdose', 285),
 ('Sprain of ankle', 266),
 ('Polyp of colon', 261),
 ("Alzheimer's disease (disorder)", 223),
 ('Fracture of forearm', 222),
 ('Fracture of the vertebral column with spinal cord injury', 205),
 ('Hyperlipidemia', 203),
 ('Osteoporosis (disorder)', 198),
 ('Malignant tumor of colon', 197),
 ('Normal pregnancy', 186),
 ('Anemia (disorder)', 182),
 ('Concussion with no loss of consciousness', 176),
 ('Overlapping malignant neopl

#### Origin Code Test

In [11]:
evalColDirvertyDIC = getOrgAllColumnDivertyCNT(originalDataSet_Path)
evalColDirvertyDIC

['AGE', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITIOIN', 'DEATH']


{0: 111.0, 1: 6, 2: 24, 3: 2, 4: 27, 5: 133}

In [12]:
org_moned_dataset = fc.readCSV('./normalized/recover_origin_wc.csv', sep='\t')
print(len(org_moned_dataset))

206290


In [13]:
org_moned_dataset[0]

['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']

In [14]:
leScore = getLEScore(org_moned_dataset)
float_to_str(leScore)

['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 1796.018181816946
AGE len : 206289
AGE colLEScore : 0.008706320656055078

BIRTHPLACE sum : 107.49999999999575
BIRTHPLACE len : 206289
BIRTHPLACE colLEScore : 0.0005211135833708814

CONDITION sum : 1072.9242424234246
CONDITION len : 206289
CONDITION colLEScore : 0.005201073457253778

ETHNICITY sum : 81.99999999999751
ETHNICITY len : 206289
ETHNICITY colLEScore : 0.00039750059382709455

GENDER sum : 0.0
GENDER len : 206289
GENDER colLEScore : 0.0

RACE sum : 8.200000000000003
RACE len : 206289
RACE colLEScore : 3.975005938271068e-05

totla LE SCORE : 0.0024776


'0.0024776'

In [15]:
berted_dataset_bert = fc.readCSV('./normalized/recover_bert_wc.csv', sep='\t')
print(len(berted_dataset_bert))

207383


In [16]:
leScor2 = getLEScore(berted_dataset_bert)
float_to_str(leScor2)

['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 0.0
AGE len : 207382
AGE colLEScore : 0.0

BIRTHPLACE sum : 994.423076923261
BIRTHPLACE len : 207382
BIRTHPLACE colLEScore : 0.004795127238252409

CONDITION sum : 3319.9393939481024
CONDITION len : 207382
CONDITION colLEScore : 0.016008811728829418

ETHNICITY sum : 4764.043478263068
ETHNICITY len : 207382
ETHNICITY colLEScore : 0.022972309449533074

GENDER sum : 0.0
GENDER len : 207382
GENDER colLEScore : 0.0

RACE sum : 2245.4000000000924
RACE len : 207382
RACE colLEScore : 0.010827362066139263

totla LE SCORE : 0.0091006


'0.0091006'

#### Test Code